In [5]:
import yfinance as yf
import pandas as 

In [39]:
sp500_tickers

['AAPL',
 'MSFT',
 'AMZN',
 'NVDA',
 'GOOGL',
 'GOOG',
 'META',
 'BRK.B',
 'TSLA',
 'JPM',
 'UNH',
 'V',
 'MA',
 'PG',
 'HD',
 'LLY',
 'AVGO',
 'CRM',
 'XOM',
 'COST',
 'ABBV',
 'MRK',
 'PEP',
 'KO',
 'ADBE',
 'CSCO',
 'WMT',
 'MCD',
 'BAC',
 'ORCL',
 'ACN',
 'ABT',
 'DHR',
 'LIN',
 'CMCSA',
 'NFLX',
 'AMD',
 'TXN',
 'INTU',
 'AMGN',
 'DIS',
 'HON',
 'VZ',
 'QCOM',
 'IBM',
 'PM',
 'NOW',
 'CAT',
 'GE',
 'SPGI',
 'ISRG',
 'UNP',
 'BKNG',
 'RTX',
 'LOW',
 'T',
 'NEE',
 'GS',
 'BLK',
 'MS',
 'UPS',
 'COP',
 'SCHW',
 'PFE',
 'AXP',
 'DE',
 'LMT',
 'ELV',
 'MDT',
 'SYK',
 'PLD',
 'ADI',
 'TJX',
 'AMAT',
 'GILD',
 'MMC',
 'VRTX',
 'CB',
 'CI',
 'BDX',
 'CME',
 'DUK',
 'SO',
 'MO',
 'ZTS',
 'TGT',
 'REGN',
 'ITW',
 'FIS',
 'CL',
 'EOG',
 'PNC',
 'NKE',
 'USB',
 'APD',
 'WM',
 'ETN',
 'CSX',
 'ICE',
 'SHW',
 'EQIX',
 'EMR',
 'GM',
 'NSC',
 'AON',
 'HUM',
 'GD',
 'MCO',
 'PGR',
 'PSX',
 'FDX',
 'ROP',
 'EW',
 'SLB',
 'KMB',
 'AIG',
 'SRE',
 'OXY',
 'MSI',
 'VLO',
 'TRV',
 'AEP',
 'PH',
 'AZO',


In [40]:
sp500_tick = sp500_tickers

In [47]:
import yfinance as yf
import pandas as pd
import sqlite3
from datetime import datetime

# ----------------------------
# CONFIG
# ----------------------------
DB_NAME = "market_data.db"
TABLE_NAME = "stock_prices"

START_DATE = "2000-01-01"
END_DATE = datetime.today().strftime("%Y-%m-%d")

ticker_relevant = sp500_tick

# ----------------------------
# SQLITE SETUP
# ----------------------------
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

cursor.execute(f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
    date TEXT NOT NULL,
    ticker TEXT NOT NULL,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER,
    PRIMARY KEY (date, ticker)
)
""")

# Performance tuning (safe)
cursor.execute("PRAGMA journal_mode=WAL;")
cursor.execute("PRAGMA synchronous=NORMAL;")

conn.commit()

# ----------------------------
# DOWNLOAD DATA
# ----------------------------
for i in sp500_tick:
    print("Downloading data from Yahoo Finance...", i)
    
    df = yf.download(
        tickers=i,
        start=START_DATE,
        end=END_DATE

    )
    
    if df.empty:
        continue
    
    # ----------------------------
    # NORMALIZE DATA
    # ----------------------------
    # Convert multi-index columns -> rows
    df = df.stack(level=1).reset_index()
    
    df.columns = [
        "date",
        "ticker",
        "open",
        "high",
        "low",
        "close",
        "volume"
    ]
    
    # Convert date to string (SQLite friendly)
    df["date"] = df["date"].astype(str)
    
    # Drop rows with no volume (market holidays, bad symbols)
    df = df.dropna(subset=["volume"])
    
    print(f"Prepared {len(df):,} rows for insertion")
    
    # ----------------------------
    # INSERT INTO SQLITE
    # ----------------------------
    insert_query = f"""
    INSERT OR IGNORE INTO {TABLE_NAME}
    (date, ticker, open, high, low, close, volume)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """
    
    cursor.executemany(
        insert_query,
        df[["date", "ticker", "open", "high", "low", "close", "volume"]].values.tolist()
    )
    
    conn.commit()
    
    
    print("✅ Data successfully stored in SQLite", i)
conn.close()

[*********************100%***********************]  1 of 1 completed
/var/folders/l1/7ycv9dtd6gvf_532by4g4bf40000gn/T/ipykernel_21265/356006273.py:62: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).reset_index()
[*********************100%***********************]  1 of 1 completed
/var/folders/l1/7ycv9dtd6gvf_532by4g4bf40000gn/T/ipykernel_21265/356006273.py:62: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).reset_index()
[*********************100%***********************]  1 of 1 completed

Prepared 6,571 rows for insertion
✅ Data successfully stored in SQLite AAPL
Prepared 6,571 rows for insertion
✅ Data successfully stored in SQLite MSFT
Prepared 6,571 rows for insertion



/var/folders/l1/7ycv9dtd6gvf_532by4g4bf40000gn/T/ipykernel_21265/356006273.py:62: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).reset_index()
[*********************100%***********************]  1 of 1 completed
/var/folders/l1/7ycv9dtd6gvf_532by4g4bf40000gn/T/ipykernel_21265/356006273.py:62: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).reset_index()
[*********************100%***********************]  1 of 1 completed
/var/folders/l1/7ycv9dtd6gvf_532by4g4bf40000gn/T/ipykernel_21265/356006273.py:62: FutureWarning: The previo

✅ Data successfully stored in SQLite AMZN
Prepared 6,571 rows for insertion
✅ Data successfully stored in SQLite NVDA
Prepared 5,409 rows for insertion
✅ Data successfully stored in SQLite GOOGL
Prepared 5,409 rows for insertion
✅ Data successfully stored in SQLite GOOG


[*********************100%***********************]  1 of 1 completed
/var/folders/l1/7ycv9dtd6gvf_532by4g4bf40000gn/T/ipykernel_21265/356006273.py:62: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).reset_index()
$BRK.B: possibly delisted; no timezone found
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BRK.B']: possibly delisted; no timezone found


...

[*********************100%***********************]  1 of 1 completed
/var/folders/l1/7ycv9dtd6gvf_532by4g4bf40000gn/T/ipykernel_21265/356006273.py:62: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).reset_index()


Prepared 6,571 rows for insertion
✅ Data successfully stored in SQLite DVN


[*********************100%***********************]  1 of 1 completed

Prepared 5,245 rows for insertion
✅ Data successfully stored in SQLite DXCM



/var/folders/l1/7ycv9dtd6gvf_532by4g4bf40000gn/T/ipykernel_21265/356006273.py:62: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).reset_index()


In [49]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("market_data.db")

pd.read_sql(
    "SELECT * FROM stock_prices WHERE ticker='SEE' ORDER BY date DESC LIMIT 5",
    conn
)

,date,ticker,open,high,low,close,volume
0,2026-02-18,SEE,41.919998,41.990002,41.869999,41.959999,3675000
1,2026-02-17,SEE,41.950001,42.000000,41.860001,41.930000,3140300
2,2026-02-13,SEE,41.930000,42.000000,41.930000,41.959999,2456400
3,2026-02-12,SEE,41.950001,42.000000,41.930000,41.959999,5027200
4,2026-02-11,SEE,41.869999,41.930000,41.849998,41.880001,1732200


In [50]:
import yfinance as yf
import sqlite3
from datetime import datetime

# ----------------------------
# CONFIG
# ----------------------------
DB_NAME = "equity_master.db"
TABLE_NAME = "equity_master"


# ----------------------------
# SQLITE SETUP
# ----------------------------
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

cursor.execute(f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
    ticker TEXT PRIMARY KEY,
    company_name TEXT,
    short_name TEXT,
    sector TEXT,
    industry TEXT,
    exchange TEXT,
    country TEXT,
    currency TEXT,
    market_cap INTEGER,
    employees INTEGER,
    website TEXT,
    business_summary TEXT,
    last_updated TEXT
)
""")

conn.commit()

# ----------------------------
# FETCH & INSERT DATA
# ----------------------------
rows = []

for ticker in sp500_tick:
    try:
        print(f"Fetching info for {ticker}...")
        t = yf.Ticker(ticker)
        info = t.info

        rows.append((
            ticker,
            info.get("longName"),
            info.get("shortName"),
            info.get("sector"),
            info.get("industry"),
            info.get("exchange"),
            info.get("country"),
            info.get("currency"),
            info.get("marketCap"),
            info.get("fullTimeEmployees"),
            info.get("website"),
            info.get("longBusinessSummary"),
            datetime.utcnow().isoformat()
        ))

    except Exception as e:
        print(f"❌ Failed for {ticker}: {e}")

# ----------------------------
# UPSERT INTO SQLITE
# ----------------------------
cursor.executemany(f"""
INSERT INTO {TABLE_NAME} (
    ticker, company_name, short_name, sector, industry,
    exchange, country, currency, market_cap, employees,
    website, business_summary, last_updated
)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
ON CONFLICT(ticker) DO UPDATE SET
    company_name=excluded.company_name,
    short_name=excluded.short_name,
    sector=excluded.sector,
    industry=excluded.industry,
    exchange=excluded.exchange,
    country=excluded.country,
    currency=excluded.currency,
    market_cap=excluded.market_cap,
    employees=excluded.employees,
    website=excluded.website,
    business_summary=excluded.business_summary,
    last_updated=excluded.last_updated
""", rows)

conn.commit()
conn.close()

print("✅ Equity master DB populated successfully")


Fetching info for AAPL...


/var/folders/l1/7ycv9dtd6gvf_532by4g4bf40000gn/T/ipykernel_21265/346176535.py:62: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime.utcnow().isoformat()


Fetching info for MSFT...
Fetching info for AMZN...
Fetching info for NVDA...
Fetching info for GOOGL...
Fetching info for GOOG...
Fetching info for META...
Fetching info for BRK.B...
Fetching info for TSLA...
Fetching info for JPM...
Fetching info for UNH...
Fetching info for V...
Fetching info for MA...
Fetching info for PG...
Fetching info for HD...
Fetching info for LLY...
Fetching info for AVGO...
Fetching info for CRM...
Fetching info for XOM...
Fetching info for COST...
Fetching info for ABBV...
Fetching info for MRK...
Fetching info for PEP...
Fetching info for KO...
Fetching info for ADBE...
Fetching info for CSCO...
Fetching info for WMT...
Fetching info for MCD...
Fetching info for BAC...
Fetching info for ORCL...
Fetching info for ACN...
Fetching info for ABT...
Fetching info for DHR...
Fetching info for LIN...
Fetching info for CMCSA...
Fetching info for NFLX...
Fetching info for AMD...
Fetching info for TXN...
Fetching info for INTU...
Fetching info for AMGN...
Fetching i

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WBA"}}}


Fetching info for HCA...
Fetching info for CPRT...
Fetching info for ORLY...
Fetching info for SNPS...
Fetching info for GWW...
Fetching info for FAST...
Fetching info for KEYS...
Fetching info for HPQ...
Fetching info for BK...
Fetching info for AMP...
Fetching info for KHC...
Fetching info for AME...
Fetching info for EA...
Fetching info for APTV...
Fetching info for PPG...
Fetching info for CTSH...
Fetching info for VRSK...
Fetching info for ODFL...
Fetching info for DOW...
Fetching info for WEC...
Fetching info for PRU...
Fetching info for EFX...
Fetching info for DD...
Fetching info for LEN...
Fetching info for IT...
Fetching info for GLW...
Fetching info for STT...
Fetching info for FITB...
Fetching info for NTRS...
Fetching info for TROW...
Fetching info for ES...
Fetching info for EBAY...
Fetching info for RSG...
Fetching info for TSCO...
Fetching info for WY...
Fetching info for EIX...
Fetching info for CHTR...
Fetching info for MTB...
Fetching info for GPN...
Fetching info fo

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ANSS"}}}


Fetching info for ETSY...
Fetching info for WAT...
Fetching info for HPE...
Fetching info for HBAN...
Fetching info for AKAM...
Fetching info for CFG...
Fetching info for ZBRA...
Fetching info for CE...
Fetching info for DRI...
Fetching info for RF...
Fetching info for STE...
Fetching info for LUV...
Fetching info for HOLX...
Fetching info for AVY...
Fetching info for PWR...
Fetching info for FMC...
Fetching info for MKC...
Fetching info for MOH...
Fetching info for COO...
Fetching info for BR...
Fetching info for EXPE...
Fetching info for UAL...
Fetching info for TRMB...
Fetching info for AES...
Fetching info for BALL...
Fetching info for IEX...
Fetching info for TYL...
Fetching info for POOL...
Fetching info for TER...
Fetching info for PKI...
Fetching info for DG...
Fetching info for WRB...
Fetching info for NDAQ...
Fetching info for MAA...
Fetching info for TXT...
Fetching info for SNA...
Fetching info for JKHY...
Fetching info for CNP...
Fetching info for LYB...
Fetching info for 

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: K"}}}


Fetching info for L...
Fetching info for IRM...
Fetching info for PFG...
Fetching info for MOS...
Fetching info for APA...
Fetching info for NRG...
Fetching info for HST...
Fetching info for LW...
Fetching info for IP...
Fetching info for UDR...
Fetching info for ROL...
Fetching info for RCL...
Fetching info for BEN...
Fetching info for EVRG...
Fetching info for FRT...
Fetching info for MGM...
Fetching info for J...
Fetching info for VTR...
Fetching info for LDOS...
Fetching info for RE...
Fetching info for KIM...
Fetching info for AAL...
Fetching info for HSIC...
Fetching info for BIO...
Fetching info for TFX...
Fetching info for CMA...
Fetching info for TPR...
Fetching info for NDSN...
Fetching info for UHS...
Fetching info for PNR...
Fetching info for BXP...
Fetching info for ALLE...
Fetching info for PEAK...
Fetching info for QRVO...
Fetching info for GNRC...
Fetching info for NI...
Fetching info for BBY...
Fetching info for EMN...
Fetching info for AAP...
Fetching info for WHR...


In [59]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("equity_master.db")

pd.read_sql(
    "SELECT industry, count(*) FROM equity_master group by industry order by industry desc",
    conn
)


,industry,count(*)
0,Waste Management,2
1,Utilities - Regulated Gas,1
2,Utilities - Regulated Electric,16
3,Utilities - Independent Power Producers,1
4,Utilities - Diversified,2
...,...,...
88,Apparel Retail,2
89,Airlines,4
90,Agricultural Inputs,2
91,Aerospace & Defense,6


In [87]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("market_data.db")

acc = pd.read_sql(
    "SELECT * FROM stock_prices WHERE ticker='SEE' ORDER BY date DESC LIMIT 5",
    conn
)

In [89]:
acc.columns

Index(['date', 'ticker', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [71]:
t = yf.Ticker("AAPL")
info = t.info

In [74]:
print(t.quarterly_balance_sheet)
print("\n\n\n\n actions:\n",t.actions)
print("\n\n\n\n splits:\n",t.splits)
print("\n\n\n\n cashflow:\n",t.cashflow)
print("\n\n\n\n dividends:\n",t.dividends)
print("\n\n\n\n recommendations:\n",t.recommendations)
print("\n\n\n\n major_holders:\n",t.major_holders)
print("\n\n\n\n mutualfund_holders:\n",t.mutualfund_holders)


                                                    2025-12-31    2025-09-30  \
Ordinary Shares Number                            1.469793e+10  1.477326e+10   
Share Issued                                      1.469793e+10  1.477326e+10   
Net Debt                                          4.519200e+10  6.272300e+10   
Total Debt                                        9.050900e+10  9.865700e+10   
Tangible Book Value                               8.819000e+10  7.373300e+10   
...                                                        ...           ...   
Cash Cash Equivalents And Short Term Investments  6.690700e+10  5.469700e+10   
Other Short Term Investments                      2.159000e+10  1.876300e+10   
Cash And Cash Equivalents                         4.531700e+10  3.593400e+10   
Cash Equivalents                                  1.449100e+10  7.667000e+09   
Cash Financial                                    3.082600e+10  2.826700e+10   

                                       

In [75]:
t.balance_sheet

,2025-09-30,2024-09-30,2023-09-30,2022-09-30,2021-09-30
Treasury Shares Number,NaN,NaN,0.000000e+00,NaN,NaN
Ordinary Shares Number,1.477326e+10,1.511679e+10,1.555006e+10,1.594342e+10,NaN
Share Issued,1.477326e+10,1.511679e+10,1.555006e+10,1.594342e+10,NaN
Net Debt,6.272300e+10,7.668600e+10,8.112300e+10,9.642300e+10,NaN
Total Debt,9.865700e+10,1.066290e+11,1.110880e+11,1.324800e+11,NaN
...,...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,5.469700e+10,6.517100e+10,6.155500e+10,4.830400e+10,NaN
Other Short Term Investments,1.876300e+10,3.522800e+10,3.159000e+10,2.465800e+10,NaN
Cash And Cash Equivalents,3.593400e+10,2.994300e+10,2.996500e+10,2.364600e+10,NaN
Cash Equivalents,7.667000e+09,2.744000e+09,1.606000e+09,5.100000e+09,NaN


In [63]:
t.actions

,Dividends,Stock Splits
Date,,
2022-06-13 00:00:00-04:00,0.0,4.0


In [67]:
t.splits

Date
2022-06-13 00:00:00-04:00    4.0
Name: Stock Splits, dtype: float64